In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap
import os
import glob
import re
from nltk.corpus import stopwords


In [3]:
df = pd.read_csv('medicine_dataset.csv')
df

C:\Users\jakem\AppData\Local\Temp\ipykernel_22996\2024557974.py:1: DtypeWarning: Columns (42,43,44,45,46,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('medicine_dataset.csv')


,id,name,substitute0,substitute1,substitute2,substitute3,substitute4,sideEffect0,sideEffect1,sideEffect2,...,sideEffect41,use0,use1,use2,use3,use4,Chemical Class,Habit Forming,Therapeutic Class,Action Class
0,1,augmentin 625 duo tablet,Penciclav 500 mg/125 mg Tablet,Moxikind-CV 625 Tablet,Moxiforce-CV 625 Tablet,Fightox 625 Tablet,Novamox CV 625mg Tablet,Vomiting,Nausea,Diarrhea,...,NaN,Treatment of Bacterial infections,NaN,NaN,NaN,NaN,NaN,No,ANTI INFECTIVES,NaN
1,2,azithral 500 tablet,Zithrocare 500mg Tablet,Azax 500 Tablet,Zady 500 Tablet,Cazithro 500mg Tablet,Trulimax 500mg Tablet,Vomiting,Nausea,Abdominal pain,...,NaN,Treatment of Bacterial infections,NaN,NaN,NaN,NaN,Macrolides,No,ANTI INFECTIVES,Macrolides
2,3,ascoril ls syrup,Solvin LS Syrup,Ambrodil-LX Syrup,Zerotuss XP Syrup,Capex LS Syrup,Broxum LS Syrup,Nausea,Vomiting,Diarrhea,...,NaN,Treatment of Cough with mucus,NaN,NaN,NaN,NaN,NaN,No,RESPIRATORY,NaN
3,4,allegra 120mg tablet,Lcfex Tablet,Etofex 120mg Tablet,Nexofex 120mg Tablet,Fexise 120mg Tablet,Histafree 120 Tablet,Headache,Drowsiness,Dizziness,...,NaN,Treatment of Sneezing and runny nose due to al...,Treatment of Allergic conditions,NaN,NaN,NaN,Diphenylmethane Derivative,No,RESPIRATORY,H1 Antihistaminics (second Generation)
4,5,avil 25 tablet,Eralet 25mg Tablet,NaN,NaN,NaN,NaN,Sleepiness,Dryness in mouth,NaN,...,NaN,Treatment of Allergic conditions,NaN,NaN,NaN,NaN,Pyridines Derivatives,No,RESPIRATORY,H1 Antihistaminics (First Generation)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248213,248214,zestrain 100mg/325mg/250mg tablet mr,Aclotec MR 100mg/325mg/250mg Tablet,SAMONEC MR TABLET,Uniclof 100mg/325mg/250mg Tablet MR,Macito 100mg/325mg/250mg Tablet MR,Rumatin 100mg/325mg/250mg Tablet MR,Nausea,Vomiting,Heartburn,...,NaN,Treatment of Muscular pain,NaN,NaN,NaN,NaN,NaN,No,PAIN ANALGESICS,NaN
248214,248215,zoxinace 200mg tablet sr,Algic 200mg Tablet SR,Topnac 200mg Tablet SR,Kindac 200mg Tablet SR,Ultranac 200mg Tablet SR,Bignac 200mg Tablet SR,Dizziness,Indigestion,Nausea,...,NaN,Pain relief,NaN,NaN,NaN,NaN,Dichlorobenzenes Derivative,No,PAIN ANALGESICS,NSAID's- Non-Selective COX 1&2 Inhibitors (ace...
248215,248216,zivex 25mg tablet,HD Zine 25mg Tablet,Hydrocas 25mg Tablet,Hyzox 25 Tablet,Hydil 25mg Tablet,Zyzine 25mg Tablet,Sedation,Nausea,Vomiting,...,NaN,Treatment of Anxiety,Treatment of Skin conditions with inflammation...,NaN,NaN,NaN,Piperazine Derivative,No,RESPIRATORY,H1 Antihistaminics (First Generation)
248216,248217,zi fast 500mg injection,Zycin 500mg Injection,Aziwok 500mg Injection,Azirow 500mg Injection,Toracin 500mg Injection,Azymed 500mg Injection,"Injection site reactions (pain, swelling, redn...",Vomiting,Nausea,...,NaN,Treatment of Bacterial infections,NaN,NaN,NaN,NaN,Macrolides,No,ANTI INFECTIVES,Macrolides


In [4]:
# Check for missing values
print(df.isnull().sum())

id                        0
name                      0
substitute0            9597
substitute1           14351
substitute2           17985
substitute3           21362
substitute4           24256
sideEffect0               0
sideEffect1            9802
sideEffect2           18718
sideEffect3           40580
sideEffect4           84658
sideEffect5          116960
sideEffect6          156361
sideEffect7          180468
sideEffect8          199712
sideEffect9          210510
sideEffect10         220944
sideEffect11         227887
sideEffect12         231936
sideEffect13         233491
sideEffect14         237799
sideEffect15         240537
sideEffect16         242209
sideEffect17         242836
sideEffect18         243703
sideEffect19         244272
sideEffect20         244995
sideEffect21         245093
sideEffect22         245170
sideEffect23         245313
sideEffect24         245495
sideEffect25         246715
sideEffect26         246715
sideEffect27         246724
sideEffect28        

In [5]:
df_summary = df[["id",
                 "name",
                 "substitute0",
                 "substitute1",
				 "sideEffect0",
				 "sideEffect1",
				 "sideEffect2",
				 "sideEffect3",
				 "sideEffect4",
				 "use0",	
				 "use1",
				"use2",
                ]]

df_summary.head()
            			

,id,name,substitute0,substitute1,sideEffect0,sideEffect1,sideEffect2,sideEffect3,sideEffect4,use0,use1,use2
0,1,augmentin 625 duo tablet,Penciclav 500 mg/125 mg Tablet,Moxikind-CV 625 Tablet,Vomiting,Nausea,Diarrhea,NaN,NaN,Treatment of Bacterial infections,NaN,NaN
1,2,azithral 500 tablet,Zithrocare 500mg Tablet,Azax 500 Tablet,Vomiting,Nausea,Abdominal pain,Diarrhea,NaN,Treatment of Bacterial infections,NaN,NaN
2,3,ascoril ls syrup,Solvin LS Syrup,Ambrodil-LX Syrup,Nausea,Vomiting,Diarrhea,Upset stomach,Stomach pain,Treatment of Cough with mucus,NaN,NaN
3,4,allegra 120mg tablet,Lcfex Tablet,Etofex 120mg Tablet,Headache,Drowsiness,Dizziness,Nausea,NaN,Treatment of Sneezing and runny nose due to al...,Treatment of Allergic conditions,NaN
4,5,avil 25 tablet,Eralet 25mg Tablet,NaN,Sleepiness,Dryness in mouth,NaN,NaN,NaN,Treatment of Allergic conditions,NaN,NaN


In [6]:
df_complete = df_summary.fillna('NA')

df_complete.head()

,id,name,substitute0,substitute1,sideEffect0,sideEffect1,sideEffect2,sideEffect3,sideEffect4,use0,use1,use2
0,1,augmentin 625 duo tablet,Penciclav 500 mg/125 mg Tablet,Moxikind-CV 625 Tablet,Vomiting,Nausea,Diarrhea,NA,NA,Treatment of Bacterial infections,NA,NA
1,2,azithral 500 tablet,Zithrocare 500mg Tablet,Azax 500 Tablet,Vomiting,Nausea,Abdominal pain,Diarrhea,NA,Treatment of Bacterial infections,NA,NA
2,3,ascoril ls syrup,Solvin LS Syrup,Ambrodil-LX Syrup,Nausea,Vomiting,Diarrhea,Upset stomach,Stomach pain,Treatment of Cough with mucus,NA,NA
3,4,allegra 120mg tablet,Lcfex Tablet,Etofex 120mg Tablet,Headache,Drowsiness,Dizziness,Nausea,NA,Treatment of Sneezing and runny nose due to al...,Treatment of Allergic conditions,NA
4,5,avil 25 tablet,Eralet 25mg Tablet,NA,Sleepiness,Dryness in mouth,NA,NA,NA,Treatment of Allergic conditions,NA,NA


In [7]:

def fix(text):
    #list of contractions
    contractions = ["ain't", "aren't", "can't", "can't've", "'cause", "could've", "couldn't", "couldn't've", "didn't", "doesn't", "don't", "hadn't", "hadn't've", "hasn't", "haven't", "he'd", "he'd've", "he'll", "he'll've", "he's", "how'd", "how'd'y", "how'll", "how's", "I'd", "I'd've", "I'll", "I'll've", "I'm", "I've", "isn't", "it'd", "it'd've", "it'll", "it'll've", "it's", "let's", "ma'am", "mayn't", "might've", "mightn't", "mightn't've", "must've", "mustn't", "mustn't've", "needn't", "needn't've", "o'clock", "oughtn't", "oughtn't've", "shan't", "sha'n't", "shan't've", "she'd", "she'd've", "she'll", "she'll've", "she's", "should've", "shouldn't", "shouldn't've", "so've", "so's", "that'd", "that'd've", "that's", "there'd", "there'd've", "there's", "they'd", "they'd've", "they'll", "they'll've", "they're", "they've", "to've", "wasn't", "we'd", "we'd've", "we'll", "we'll've", "we're", "we've", "weren't", "what'll", "what'll've", "what're", "what's", "what've", "when's", "when've", "where'd", "where's", "where've", "who'll", "who'll've", "who's", "who've", "why's", "why've", "will've", "won't", "won't've", "would've", "wouldn't", "wouldn't've", "y'all", "y'all'd", "y'all'd've", "y'all're", "y'all've", "you'd", "you'd've", "you'll", "you'll've", "you're", "you've"]

    contractions_dict = {c: " ".join(c.split("'")) for c in contractions}
    contractions_dict = {
        "ain't": "am not / are not / is not / has not / have not",
        "aren't": "are not / am not",
        "can't": "cannot",
        "can't've": "cannot have",
        "'cause": "because",
        "could've": "could have",
        "couldn't": "could not",
        "couldn't've": "could not have",
        "didn't": "did not",
        "doesn't": "does not",
        "don't": "do not",
        "hadn't": "had not",
        "hadn't've": "had not have",
        "hasn't": "has not",
        "haven't": "have not",
        "he'd": "he had / he would",
        "he'd've": "he would have",
        "he'll": "he shall / he will",
        "he'll've": "he shall have / he will have",
        "he's": "he has / he is",
        "how'd": "how did",
        "how'd'y": "how do you",
        "how'll": "how will",
        "how's": "how has / how is / how does",
        "I'd": "I had / I would",
        "I'd've": "I would have",
        "I'll": "I shall / I will",
        "I'll've": "I shall have / I will have",
        "I'm": "I am",
        "I've": "I have",
        "isn't": "is not",
        "it'd": "it had / it would",
        "it'd've": "it would have",
        "it'll": "it shall / it will",
        "it'll've": "it shall have / it will have",
        "it's": "it has / it is",
        "let's": "let us",
        "might've": "might have",
        "mightn't": "might not",
        "mightn't've": "might not have",
        "must've": "must have",
        "mustn't": "must not",
        "mustn't've": "must not have",
        "needn't": "need not",
        "needn't've": "need not have",
        "o'clock": "of the clock",
        "she'd": "she had / she would",
        "she'd've": "she would have",
        "she'll": "she shall / she will",
        "she'll've": "she shall have / she will have",
        "she's": "she has / she is",
        "should've": "should have",
        "shouldn't": "should not",
        "shouldn't've": "should not have",
        "so've": "so have",
        "so's": "so as / so is",
        "that'd": "that would / that had",
        "that's": "that has / that is",
        "there'd": "there had / there would",
        "there'd've": "there would have",
        "there's": "there has / there is",
        "they'd": "they had / they would",
        "they'd've": "they would have",
        "they'll": "they shall / they will",
        "they're": "they are",
        "they've": "they have",
        "wasn't": "was not",
        "we'd": "we had / we would",
        "we'll": "we will",
        "we're": "we are",
        "we've": "we have",
        "weren't": "were not",
        "what'll": "what shall / what will",
        "what's": "what has / what is",
        "what've": "what have",
        "when's": "when has / when is",
        "when've": "when have",
        "where'd": "where did",
        "where's": "where has / where is",
        "where've": "where have",
        "who'll": "who shall / who will",
        "who's": "who has / who is",
        "who've": "who have",
        "why's": "why has / why is",
        "why've": "why have",
        "will've": "will have",
        "won't": "will not",
        "won't've": "will not have",
        "would've": "would have",
        "wouldn't": "would not",
        "y'all": "you all",
        "you'd": "you had / you would",
        "you'll": "you shall / you will",
        "you're": "you are",
        "you've": "you have"
        }

    contractions_re = re.compile('(%s)' % '|'.join(contractions_dict.keys()))

    def replace(match):
        return contractions_dict[match.group(0)]

    return contractions_re.sub(replace, text)


In [8]:
# Define stop_words variable
stop_words = set(stopwords.words('english'))

# Define contractions dictionary
contractions_dict = {
    "ain't": "is not",
    "aren't": "are not",
    # Add more contractions here...
}

# Define contractions regular expression and replacement function
contractions_re = re.compile('(%s)' % '|'.join(contractions_dict.keys()))

def replace(match):
    return contractions_dict[match.group(0)]

def normalize_text(text):
    # Ensure text is a string
    text = str(text)
    # Convert text to lowercase
    text = text.lower()
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove stopwords
    tokens = text.split()  # Split text into tokens
    tokens = [word for word in tokens if word not in stop_words]
    # Expand contractions tions)
    text = contractions_re.sub(replace, text)
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Standardize spelling
    text = text.replace('colour', 'color')
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Apply normalize_text function to df_complete
df_complete[["name", "substitute0", "substitute1", "sideEffect0", "sideEffect1", "sideEffect2", "sideEffect3", "sideEffect4", "use0", "use1", "use2"]].applymap(normalize_text)

C:\Users\jakem\AppData\Local\Temp\ipykernel_22996\351874125.py:38: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_complete[["name", "substitute0", "substitute1", "sideEffect0", "sideEffect1", "sideEffect2", "sideEffect3", "sideEffect4", "use0", "use1", "use2"]].applymap(normalize_text)


,name,substitute0,substitute1,sideEffect0,sideEffect1,sideEffect2,sideEffect3,sideEffect4,use0,use1,use2
0,augmentin duo tablet,penciclav mg mg tablet,moxikindcv tablet,vomiting,nausea,diarrhea,na,na,treatment of bacterial infections,na,na
1,azithral tablet,zithrocare mg tablet,azax tablet,vomiting,nausea,abdominal pain,diarrhea,na,treatment of bacterial infections,na,na
2,ascoril ls syrup,solvin ls syrup,ambrodillx syrup,nausea,vomiting,diarrhea,upset stomach,stomach pain,treatment of cough with mucus,na,na
3,allegra mg tablet,lcfex tablet,etofex mg tablet,headache,drowsiness,dizziness,nausea,na,treatment of sneezing and runny nose due to al...,treatment of allergic conditions,na
4,avil tablet,eralet mg tablet,na,sleepiness,dryness in mouth,na,na,na,treatment of allergic conditions,na,na
...,...,...,...,...,...,...,...,...,...,...,...
248213,zestrain mgmgmg tablet mr,aclotec mr mgmgmg tablet,samonec mr tablet,nausea,vomiting,heartburn,stomach pain,diarrhea,treatment of muscular pain,na,na
248214,zoxinace mg tablet sr,algic mg tablet sr,topnac mg tablet sr,dizziness,indigestion,nausea,abdominal pain,increased liver enzymes,pain relief,na,na
248215,zivex mg tablet,hd zine mg tablet,hydrocas mg tablet,sedation,nausea,vomiting,upset stomach,constipation,treatment of anxiety,treatment of skin conditions with inflammation...,na
248216,zi fast mg injection,zycin mg injection,aziwok mg injection,injection site reactions pain swelling redness,vomiting,nausea,abdominal pain,diarrhea,treatment of bacterial infections,na,na


In [9]:
# Define stop_words variable
stop_words = set(stopwords.words('english'))




print(df_complete[["id", "name", "substitute0", "sideEffect1","use0", "use1"]].apply(normalize_text).head()) # apply the function to the df


id                                      name id length dtype int
name           augmentin duo tablet azithral tablet ascoril l...
substitute0    penciclav mg mg tablet zithrocare mg tablet so...
sideEffect1    nausea nausea vomiting drowsiness dryness in m...
use0           treatment of bacterial infections treatment of...
dtype: object
